## Clustering: grouping observations together
聚类，将观测数据分类

聚类可以解决的问题举例：
还是之前的iris数据集，如我们已经知道有3种类型的iris花，但是并没有方法对数据集中的样品进行分类：这时候我们可以尝试聚类的方法，将所有的观测数据分割为具有较明显特征的群组中。

### K-means clustering
目前存在很多不同的聚类标准和关联算法。最简单的聚类算法是K-means。


In [1]:
from sklearn import cluster,datasets
iris=datasets.load_iris()

In [2]:
X_iris=iris.data
y_iris=iris.target

In [3]:
k_means=cluster.KMeans(n_clusters=3)
k_means.fit(X_iris)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [4]:
k_means.labels_[::10]

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [5]:
print(y_iris[::10])

[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]


警告：算法并不能保证绝对分类正确。首先，确定数据样本有多少个种类很难。其次，算法对于初始化的十分敏感的，可能陷入局部最小值陷阱，尽管scikit-learn采用了一些方法来减轻这些问题的影响。

所以说，不要过度解读聚类的结果~

#### 应用实例：vector quantization，矢量量化？
一般来说，使用聚类和Kmeans方法，可以视为选取较少的样本来压缩信息量的一种方式。这种问题有时称作vector quantization。举例来说，可以应用到对图片进行色调分离。


In [6]:
import scipy as sp
try:
    face=sp.face(gray=True)
except AttributeError:
    from scipy import misc
    face=misc.face(gray=True)
X=face.reshape((-1,1))
k_means=cluster.KMeans(n_clusters=5,n_init=1)
k_means.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [8]:
import numpy as np
values=k_means.cluster_centers_.squeeze()
labels=k_means.labels_
face_compressed=np.choose(labels,values)
face_compressed.shape=face.shape

### Hierarchical agglomerative clustering: Ward
分层聚类

分层聚类方法是一种聚类分析，旨在构建聚类的层次结构。一般来说，这种技术的一些方法包括：
1、Agglomerative—凝聚—bottom-up由下而上的方法：每个样本始于它本身的聚类，并且聚类以最小化一个linkage标准的方法迭代合并。当仅有较少的观测数据的时候，这种方法尤其有趣。当聚类数量较大的时候，它比k-means在计算上更有效率。
2、Divisive—分裂—top-down由上而下的方法：所有的观测数据从一个聚类出发，然后不断向下迭代分裂。当预测较多数量的聚类时，这种方法又慢，统计上来讲也是病态的。

Connectivity-constrained clustering
使用Agglomerative聚类方法，可以通过给出连通性图来指定哪些样本可以聚集在一起。scikit中的图由它们的邻接矩阵表示。经常，使用稀疏性矩阵。例如，这可以用于在对图像进行聚类时检索连接区域。


In [9]:
from sklearn.feature_extraction.image import grid_to_graph
from sklearn.cluster import AgglomerativeClustering

# Generate data
try:  # SciPy >= 0.16 have face in misc
    from scipy.misc import face
    face = face(gray=True)
except ImportError:
    face = sp.face(gray=True)

# Resize it to 10% of the original size to speed up the processing
face = sp.misc.imresize(face, 0.10) / 255.

X = np.reshape(face, (-1, 1))

# Define the structure A of the data. Pixels connected to their neighbors.
connectivity = grid_to_graph(*face.shape)


c:\py35\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


Feature agglomeration
我们已经知道，稀疏性可以减轻维度诅咒，比如，与特征的数量相比，观察量不足。另一个方法是合并类似的特征：feature agglomeration。这个方法可以通过在特征方向的聚类来实现，也就是说转置数据后再聚类。



In [11]:
digits=datasets.load_digits()
images=digits.images
X=np.reshape(images,(len(images),-1))
connectivity=grid_to_graph(*images[0].shape)
agglo=cluster.FeatureAgglomeration(connectivity=connectivity,n_clusters=32)
agglo.fit(X)

FeatureAgglomeration(affinity='euclidean', compute_full_tree='auto',
           connectivity=<64x64 sparse matrix of type '<class 'numpy.int32'>'
	with 288 stored elements in COOrdinate format>,
           linkage='ward', memory=None, n_clusters=32,
           pooling_func=<function mean at 0x000002A1A73A5F28>)

In [12]:
X_reduced=agglo.transform(X)
X_approx=agglo.inverse_transform(X_reduced)
images_approx=np.reshape(X_approx,images.shape)


## Decompositions: from a signal to components and loadings
分解：从一个信号到组件和装载

如果X是我们的多元数据，那么我们要去解决的问题即为，在一个不同观测角度重写X：我们想学习到loading L和一系列成分C，满足X=LC。目前存在的不同用于选择成分的标准。

### Principal component analysis: PCA
主成分分析
PCA选择信号中可以解释最大方法的连续成分变量。

上述观察所涵盖的点云在一个方向上非常平坦：三个单变量特征之一几乎可以使用另外两个特征精确计算。PCA找到数据不平坦的方向。
当用于变换数据时，PCA可以通过在主子空间上投影来降低数据的维数。


In [13]:
>>> # Create a signal with only 2 useful dimensions
>>> x1 = np.random.normal(size=100)
>>> x2 = np.random.normal(size=100)
>>> x3 = x1 + x2
>>> X = np.c_[x1, x2, x3]

>>> from sklearn import decomposition
>>> pca = decomposition.PCA()
>>> pca.fit(X)


PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [14]:
>>> print(pca.explained_variance_)  

[  3.01497565e+00   1.10035180e+00   7.57434866e-32]


In [15]:
>>> # As we can see, only the 2 first components are useful
>>> pca.n_components = 2
>>> X_reduced = pca.fit_transform(X)
>>> X_reduced.shape


(100, 2)

### Independent Component Analysis: ICA
独立成分分析

ICA选择的成分，使得它们的Loading分配携带最大量的独立信息
ICA可以恢复非高斯分布的独立信号。



In [16]:
>>> # Generate sample data
>>> import numpy as np
>>> from scipy import signal
>>> time = np.linspace(0, 10, 2000)
>>> s1 = np.sin(2 * time)  # Signal 1 : sinusoidal signal
>>> s2 = np.sign(np.sin(3 * time))  # Signal 2 : square signal
>>> s3 = signal.sawtooth(2 * np.pi * time)  # Signal 3: saw tooth signal
>>> S = np.c_[s1, s2, s3]
>>> S += 0.2 * np.random.normal(size=S.shape)  # Add noise
>>> S /= S.std(axis=0)  # Standardize data
>>> # Mix data
>>> A = np.array([[1, 1, 1], [0.5, 2, 1], [1.5, 1, 2]])  # Mixing matrix
>>> X = np.dot(S, A.T)  # Generate observations

>>> # Compute ICA
>>> ica = decomposition.FastICA()
>>> S_ = ica.fit_transform(X)  # Get the estimated sources
>>> A_ = ica.mixing_.T
>>> np.allclose(X,  np.dot(S_, A_) + ica.mean_)


True